Importando bibliotecas para o desenvolvimento do projeto

In [15]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score, classification_report

In [2]:
dados = pd.read_csv('Base_Dados/credit_card.csv', sep=',')

Context:
It is important that credit card companies are able to recognize fraudulent credit card transactions so that customers are not charged for items that they did not purchase.

Content:
The dataset contains transactions made by credit cards in September 2013 by European cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-sensitive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.

https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data

In [3]:
dados.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284806 entries, 0 to 284805
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284806 non-null  float64
 1   V1      284806 non-null  float64
 2   V2      284806 non-null  float64
 3   V3      284806 non-null  float64
 4   V4      284806 non-null  float64
 5   V5      284806 non-null  float64
 6   V6      284806 non-null  float64
 7   V7      284806 non-null  float64
 8   V8      284806 non-null  float64
 9   V9      284806 non-null  float64
 10  V10     284806 non-null  float64
 11  V11     284806 non-null  float64
 12  V12     284806 non-null  float64
 13  V13     284806 non-null  float64
 14  V14     284806 non-null  float64
 15  V15     284806 non-null  float64
 16  V16     284806 non-null  float64
 17  V17     284806 non-null  float64
 18  V18     284806 non-null  float64
 19  V19     284806 non-null  float64
 20  V20     284806 non-null  float64
 21  V21     28

In [5]:
dados.describe().round(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284806.00,284806.00,284806.00,284806.00,284806.00,284806.00,284806.00,284806.00,284806.00,284806.00,...,284806.00,284806.00,284806.00,284806.00,284806.00,284806.00,284806.00,284806.00,284806.00,284806.00
mean,94813.59,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,88.35,0.00
std,47488.00,1.96,1.65,1.52,1.42,1.38,1.33,1.24,1.19,1.10,...,0.73,0.73,0.62,0.61,0.52,0.48,0.40,0.33,250.12,0.04
min,0.00,-56.41,-72.72,-48.33,-5.68,-113.74,-26.16,-43.56,-73.22,-13.43,...,-34.83,-10.93,-44.81,-2.84,-10.30,-2.60,-22.57,-15.43,0.00,0.00
25%,54201.25,-0.92,-0.60,-0.89,-0.85,-0.69,-0.77,-0.55,-0.21,-0.64,...,-0.23,-0.54,-0.16,-0.35,-0.32,-0.33,-0.07,-0.05,5.60,0.00
50%,84691.50,0.02,0.07,0.18,-0.02,-0.05,-0.27,0.04,0.02,-0.05,...,-0.03,0.01,-0.01,0.04,0.02,-0.05,0.00,0.01,22.00,0.00
75%,139320.00,1.32,0.80,1.03,0.74,0.61,0.40,0.57,0.33,0.60,...,0.19,0.53,0.15,0.44,0.35,0.24,0.09,0.08,77.16,0.00
max,172788.00,2.45,22.06,9.38,16.88,34.80,73.30,120.59,20.01,15.59,...,27.20,10.50,22.53,4.58,7.52,3.52,31.61,33.85,25691.16,1.00


In [6]:
dados.shape

(284806, 31)

In [7]:
dados.corr(method='pearson').round(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
Time,1.0000,0.1174,-0.0106,-0.4196,-0.1053,0.1731,-0.0630,0.0847,-0.0369,-0.0087,...,0.0447,0.1441,0.0511,-0.0162,-0.2331,-0.0414,-0.0051,-0.0094,-0.0106,-0.0123
V1,0.1174,1.0000,-0.0000,0.0000,-0.0000,-0.0000,-0.0000,0.0000,-0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000,0.0000,-0.2277,-0.1013
V2,-0.0106,-0.0000,1.0000,0.0000,-0.0000,-0.0000,-0.0000,0.0000,-0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000,0.0000,-0.5314,0.0913
V3,-0.4196,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,-0.0000,...,-0.0000,-0.0000,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,-0.2109,-0.1930
V4,-0.1053,-0.0000,-0.0000,0.0000,1.0000,-0.0000,-0.0000,0.0000,-0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000,0.0000,0.0987,0.1334
V5,0.1731,-0.0000,-0.0000,0.0000,-0.0000,1.0000,-0.0000,0.0000,-0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000,0.0000,-0.3864,-0.0950
V6,-0.0630,-0.0000,-0.0000,0.0000,-0.0000,-0.0000,1.0000,0.0000,-0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000,0.0000,0.2160,-0.0436
V7,0.0847,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,-0.0000,...,-0.0000,-0.0000,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,0.3973,-0.1873
V8,-0.0369,-0.0000,-0.0000,0.0000,-0.0000,-0.0000,-0.0000,0.0000,1.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0000,-0.0000,0.0000,-0.1031,0.0199
V9,-0.0087,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,1.0000,...,-0.0000,-0.0000,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,-0.0442,-0.0977


In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


print(dados.corr(method='pearson').round(4))

          Time      V1      V2      V3      V4      V5      V6      V7  \
Time    1.0000  0.1174 -0.0106 -0.4196 -0.1053  0.1731 -0.0630  0.0847   
V1      0.1174  1.0000 -0.0000  0.0000 -0.0000 -0.0000 -0.0000  0.0000   
V2     -0.0106 -0.0000  1.0000  0.0000 -0.0000 -0.0000 -0.0000  0.0000   
V3     -0.4196  0.0000  0.0000  1.0000  0.0000  0.0000  0.0000 -0.0000   
V4     -0.1053 -0.0000 -0.0000  0.0000  1.0000 -0.0000 -0.0000  0.0000   
V5      0.1731 -0.0000 -0.0000  0.0000 -0.0000  1.0000 -0.0000  0.0000   
V6     -0.0630 -0.0000 -0.0000  0.0000 -0.0000 -0.0000  1.0000  0.0000   
V7      0.0847  0.0000  0.0000 -0.0000  0.0000  0.0000  0.0000  1.0000   
V8     -0.0369 -0.0000 -0.0000  0.0000 -0.0000 -0.0000 -0.0000  0.0000   
V9     -0.0087  0.0000  0.0000 -0.0000  0.0000  0.0000  0.0000 -0.0000   
V10     0.0306 -0.0000 -0.0000  0.0000 -0.0000 -0.0000 -0.0000  0.0000   
V11    -0.2477 -0.0000 -0.0000  0.0000 -0.0000 -0.0000 -0.0000  0.0000   
V12     0.1243 -0.0000 -0.0000  0.0000

In [9]:
y = dados['Class']

In [10]:
X = dados[['Amount','V4','V11']]

In [11]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=85)

In [12]:
print(
    X_treino.shape,
    X_teste.shape,
    y_treino.shape,
    y_teste.shape,
    (X_treino.shape[0] + X_teste.shape[0])
)

(199364, 3) (85442, 3) (199364,) (85442,) 284806


Decision Tree Classifier

In [13]:
modelo_arvore = DecisionTreeClassifier(random_state=85)

In [14]:
modelo_arvore.fit(X_treino, y_treino)

DecisionTreeClassifier(random_state=85)

In [16]:
y_pred = modelo_arvore.predict(X_teste)
acuracia = accuracy_score(y_teste, y_pred)
relatorio_classificacao = classification_report(y_teste, y_pred)

print(f'Acurácia do modelo: {acuracia}')
print(f'Relatório de classificação:\n{relatorio_classificacao}')

Acurácia do modelo: 0.9985253154186465
Relatório de classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85310
           1       0.52      0.62      0.57       132

    accuracy                           1.00     85442
   macro avg       0.76      0.81      0.78     85442
weighted avg       1.00      1.00      1.00     85442



KNeighborsClassifier

In [17]:
modelo_vizinhos = KNeighborsClassifier(n_neighbors=3)

In [18]:
modelo_vizinhos.fit(X_treino, y_treino)

KNeighborsClassifier(n_neighbors=3)

In [19]:
y_pred = modelo_vizinhos.predict(X_teste)
acuracia = accuracy_score(y_teste, y_pred)
relatorio_classificacao = classification_report(y_teste, y_pred)

print(f'Acurácia do modelo: {acuracia}')
print(f'Relatório de classificação:\n{relatorio_classificacao}')

Acurácia do modelo: 0.9990051730998806
Relatório de classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85310
           1       0.86      0.42      0.57       132

    accuracy                           1.00     85442
   macro avg       0.93      0.71      0.78     85442
weighted avg       1.00      1.00      1.00     85442



SVC Linear

In [20]:
modelo_svc = LinearSVC(max_iter=10000,dual=False,random_state=85)

In [21]:
modelo_svc.fit(X_treino, y_treino)

LinearSVC(dual=False, max_iter=10000, random_state=85)

In [22]:
y_pred = modelo_svc.predict(X_teste)
acuracia = accuracy_score(y_teste, y_pred)
relatorio_classificacao = classification_report(y_teste, y_pred)

print(f'Acurácia do modelo: {acuracia}')
print(f'Relatório de classificação:\n{relatorio_classificacao}')

Acurácia do modelo: 0.9988647269492755
Relatório de classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85310
           1       0.86      0.32      0.46       132

    accuracy                           1.00     85442
   macro avg       0.93      0.66      0.73     85442
weighted avg       1.00      1.00      1.00     85442

